<a href="https://colab.research.google.com/github/Venkat3103/fantasy-football/blob/main/Compute_Teams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##INSTALL / IMPORT REQUIRED LIBRARIES

In [1]:
!pip install pulp

     |████████████████████████████████| 40.6MB 74kB/s 


In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog
from scipy.stats import linregress
from pulp import *

In [ ]:
#from google.colab import files
#uploaded = files.upload()

Saving EPL_2020_21.csv to EPL_2020_21.csv


In [ ]:
#import io
#epl=pd.read_csv(io.BytesIO(uploaded['EPL_2020_21.csv']))

# IMPORT DATA SETS FROM LOCAL COMPUTER

## Import League Dataset

In [67]:
from google.colab import files
uploaded = files.upload()

Saving ENGvsGER.xlsx to ENGvsGER (1).xlsx


In [68]:
import io
league_data=pd.read_excel(io.BytesIO(uploaded['ENGvsGER.xlsx']))
league_data

,name,pos_details,total_shots,shots_on_target,total_passes,key_passes,appearances,assists,tackles_won,interceptions,saves,red_cards,yellow_cards,own_goals,total_goals,tournament,mins_played,country
0,Harry Kane,striker,137,53,908,49,35,14,22,11,0,0,1,0,23,Premier League,3087,ENG
1,Kevin De Bruyne,midfielder,80,26,1265,80,25,12,34,9,0,0,1,0,6,Premier League,2001,NaN
2,Jack Grealish,midfielder,50,19,996,81,26,10,31,12,0,0,6,0,6,Premier League,2184,ENG
3,Bruno Fernandes,midfielder,121,51,2112,95,37,12,57,26,0,0,6,0,18,Premier League,3110,NaN
4,Tomas Soucek,midfielder,64,23,1607,18,38,1,78,62,0,1,7,1,10,Premier League,3420,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1616,Unai Dufur,midfielder,0,0,33,0,1,0,2,1,0,0,0,0,0,LaLiga,63,NaN
1617,Eñaut Mendía,midfielder,1,0,9,0,1,0,2,0,0,0,1,0,0,LaLiga,62,NaN
1618,Jokin Ezkieta,goalkeeper,0,0,19,0,1,0,0,0,1,0,0,0,0,LaLiga,90,NaN
1619,Kevin Volland,striker,56,30,755,32,35,7,33,4,0,0,4,0,16,Ligue1,2789,GER


## Import Dream11 Dataset

In [69]:
from google.colab import files
uploaded = files.upload()

Saving ENGvsGER_DREAM11.xlsx to ENGvsGER_DREAM11 (1).xlsx


In [70]:
dream11_data=pd.read_excel(io.BytesIO(uploaded['ENGvsGER_DREAM11.xlsx']))
dream_name=dream11_data.name.tolist()
dream_name


['Jordan Pickford',
 'Manuel Neuer',
 'Aaron Ramsdale',
 'Bernd Leno',
 'Kevin Trapp',
 'Sam Johnstone',
 'Robin Gosens',
 'John Stones',
 'Mats Hummels',
 'Luke Shaw',
 'Tyrone Mings',
 'Matthias Ginter',
 'Kyle Walker',
 'Antonio Rüdiger',
 'Reece James',
 'Harry Maguire',
 'Kieran Trippier',
 'Marcel Halstenberg',
 'Niklas Süle',
 'Ben Chilwell',
 'Ben White',
 'Christian Günter',
 'Conor Coady',
 'Lukas Klostermann',
 'Robin Koch',
 'Toni Kroos',
 'Joshua Kimmich',
 'Mason Mount',
 'Thomas Müller',
 'Leroy Sané',
 'Phil Foden',
 'Jadon Sancho',
 'Kalvin Phillips',
 'Leon Goretzka',
 'Ilkay Gündogan',
 'Declan Rice',
 'Bukayo Saka',
 'Emre Can',
 'Jack Grealish',
 'Jordan Henderson',
 'Florian Neuhaus',
 'Jude Bellingham',
 'Jamal Musiala',
 'Jonas Hofmann',
 'Raheem Sterling',
 'Harry Kane',
 'Kai Havertz',
 'Serge Gnabry',
 'Timo Werner',
 'Marcus Rashford',
 'Kevin Volland',
 'Dominic Calvert-Lewin']

In [245]:
dream11_data

,name,epl_points,epl_value,pos_detail,club,mat_played,is_playing,mins_played,actual_ppm
0,Jordan Pickford,108,8.5,goalkeeper,ENG,3,1,270,45
1,Manuel Neuer,33,8.5,goalkeeper,GER,3,1,270,17
2,Aaron Ramsdale,0,8.0,goalkeeper,ENG,0,0,0,0
3,Bernd Leno,0,8.0,goalkeeper,GER,0,0,0,0
4,Kevin Trapp,0,8.0,goalkeeper,GER,0,0,0,0
5,Sam Johnstone,0,8.0,goalkeeper,ENG,0,0,0,0
6,Robin Gosens,178,9.0,defender,GER,3,1,231,4
7,John Stones,130,9.0,defender,ENG,3,1,259,38
8,Mats Hummels,118,8.5,defender,GER,3,1,243,19
9,Luke Shaw,96,8.5,defender,ENG,2,1,180,56


## Assign Clubs for the day's game

In [246]:
clubs = dream11_data.club.unique()
club1 = clubs[0]
club2 = clubs[1]

##Obtain Particular Match From League Dataset

In [247]:
league_data=league_data[(league_data['country']==club1)|(league_data['country']==club2)]
league_data.reset_index(drop=True,inplace=True)
#final_data['ppm']=final_data['dream_points']/final_data['appearances']
league_data

,name,pos_details,total_shots,shots_on_target,total_passes,key_passes,appearances,assists,tackles_won,interceptions,saves,red_cards,yellow_cards,own_goals,total_goals,tournament,mins_played,country,dream_points
0,Harry Kane,striker,137,53,908,49,35,14,22,11,0,0,1,0,23,Premier League,3087,ENG,3216.6
1,Jack Grealish,midfielder,50,19,996,81,26,10,31,12,0,0,6,0,6,Premier League,2184,ENG,1694.2
2,Mason Mount,midfielder,82,27,1702,87,36,5,75,31,0,0,2,0,6,Premier League,2892,ENG,2374.4
3,Ilkay Gündogan,midfielder,54,22,1714,42,28,2,25,20,0,0,1,0,13,Premier League,2031,GER,1702.8
4,Raheem Sterling,striker,70,28,1113,39,31,7,21,15,0,0,4,0,10,Premier League,2537,ENG,1916.6
5,Dominic Calvert-Lewin,striker,83,46,675,18,33,0,17,6,0,0,3,0,16,Premier League,2876,ENG,2385.0
6,Phil Foden,midfielder,45,20,762,36,28,5,22,11,0,0,0,0,9,Premier League,1614,ENG,1427.4
7,Marcus Rashford,striker,79,37,1189,43,37,9,18,8,0,0,4,0,11,Premier League,2928,ENG,2363.8
8,Harry Maguire,defender,36,10,2227,7,34,1,29,60,0,0,11,0,2,Premier League,3048,ENG,1261.4
9,John Stones,defender,15,5,1727,2,22,0,21,18,0,1,0,0,4,Premier League,1934,ENG,727.4


##Calculate Dream 11 Points From League Data

ATTACK:
Goal
Scored by a striker
+40
Scored by a mid-fielder
+50
Scored by a defender or goalkeeper
+60
Assist
+20
Shot on Target
Includes Goals
+6
Chance Created
The final pass leading to a shot (on target including goals, blocked or off target)
+3
5 Passes Completed
+1
DEFENSE:
Tackle Won
+4
Interception Won
+4
Saves
GK
+6
Penalty Saved
GK
+50
Clean Sheet **
GK/DEF (Played +55 mins)
+20
OTHERS:
Captain
2x
Vice-Captain
1.5x
In Starting 11
+4
Coming on as a Substitute
+2
PENALTIES:
Yellow card
-4
Red card
-10
Own goal
-8
Goals conceded
GK/DEF (on the field when the goal is scored)
-2
Penalty missed 
-20


In [289]:
#true data is match data created using the league data
#final_league_data is the match data with the past league data computed over the true_data
league_data['dream_points']=0

league_data['dream_points']=league_data['dream_points']+league_data['shots_on_target']*40+league_data['assists']*20+league_data['shots_on_target']*6+(league_data['key_passes']+league_data['assists'])*3+(league_data['total_passes']/5)*1+league_data['tackles_won']*4+league_data['interceptions']*4+league_data['yellow_cards']*(-4)+league_data['red_cards']*(-10)+league_data['own_goals']*(-8)

final_league_data = pd.DataFrame()
df_s=league_data[league_data['pos_details']=="striker"]

df_m=league_data[league_data['pos_details']=="midfielder"]
df_d=league_data[(league_data['pos_details']=="defender")|(league_data['pos_details']=="goalkeeper")]
df_s['dream_points']=(df_s['dream_points']+df_s['total_goals']*40)
df_m['dream_points']=(df_m['dream_points']+df_m['total_goals']*50)
df_d['dream_points']=(df_d['dream_points']+df_d['total_goals']*60)

final_league_data=pd.concat([final_league_data,df_s])
final_league_data=pd.concat([final_league_data,df_m])
final_league_data=pd.concat([final_league_data,df_d])
final_league_data.reset_index(inplace=True,drop=True)

final_league_data


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

,name,pos_details,total_shots,shots_on_target,total_passes,key_passes,appearances,assists,tackles_won,interceptions,saves,red_cards,yellow_cards,own_goals,total_goals,tournament,mins_played,country,dream_points
0,Harry Kane,striker,137,53,908,49,35,14,22,11,0,0,1,0,23,Premier League,3087,ENG,4136.6
1,Raheem Sterling,striker,70,28,1113,39,31,7,21,15,0,0,4,0,10,Premier League,2537,ENG,2316.6
2,Dominic Calvert-Lewin,striker,83,46,675,18,33,0,17,6,0,0,3,0,16,Premier League,2876,ENG,3025.0
3,Marcus Rashford,striker,79,37,1189,43,37,9,18,8,0,0,4,0,11,Premier League,2928,ENG,2803.8
4,Timo Werner,striker,79,32,779,36,35,8,20,8,0,0,2,0,6,Premier League,2608,GER,2263.8
5,Thomas Müller,striker,51,18,1391,93,32,18,49,21,0,0,0,0,11,Bundesliga,2683,GER,2519.2
6,Serge Gnabry,striker,59,27,764,39,27,2,39,26,0,0,4,0,10,Bundesliga,1654,GER,2201.8
7,Leroy Sané,striker,57,21,732,33,32,9,18,9,0,0,2,0,6,Bundesliga,1807,GER,1758.4
8,Kevin Volland,striker,56,30,755,32,35,7,33,4,0,0,4,0,16,Ligue1,2789,GER,2560.0
9,Jack Grealish,midfielder,50,19,996,81,26,10,31,12,0,0,6,0,6,Premier League,2184,ENG,1994.2


# Merge Dataframes

In [249]:
#dream11_data
final_league_data

,name,pos_details,total_shots,shots_on_target,total_passes,key_passes,appearances,assists,tackles_won,interceptions,saves,red_cards,yellow_cards,own_goals,total_goals,tournament,mins_played,country,dream_points
0,Harry Kane,striker,137,53,908,49,35,14,22,11,0,0,1,0,23,Premier League,3087,ENG,4136.6
1,Raheem Sterling,striker,70,28,1113,39,31,7,21,15,0,0,4,0,10,Premier League,2537,ENG,2316.6
2,Dominic Calvert-Lewin,striker,83,46,675,18,33,0,17,6,0,0,3,0,16,Premier League,2876,ENG,3025.0
3,Marcus Rashford,striker,79,37,1189,43,37,9,18,8,0,0,4,0,11,Premier League,2928,ENG,2803.8
4,Timo Werner,striker,79,32,779,36,35,8,20,8,0,0,2,0,6,Premier League,2608,GER,2263.8
5,Thomas Müller,striker,51,18,1391,93,32,18,49,21,0,0,0,0,11,Bundesliga,2683,GER,2519.2
6,Serge Gnabry,striker,59,27,764,39,27,2,39,26,0,0,4,0,10,Bundesliga,1654,GER,2201.8
7,Leroy Sané,striker,57,21,732,33,32,9,18,9,0,0,2,0,6,Bundesliga,1807,GER,1758.4
8,Kevin Volland,striker,56,30,755,32,35,7,33,4,0,0,4,0,16,Ligue1,2789,GER,2560.0
9,Jack Grealish,midfielder,50,19,996,81,26,10,31,12,0,0,6,0,6,Premier League,2184,ENG,1994.2


In [250]:
temp_data = pd.merge(dream11_data,final_league_data,on='name',how='left');
#temp_data
final_data = temp_data[['name','epl_value','pos_detail','club','mins_played_x','epl_points','mins_played_y','dream_points',"is_playing"]]
final_data.rename(columns = {'mins_played_x' : 'mins_played_dream11' , 'mins_played_y' : 'mins_played_league', 'epl_points' : 'dream11_points', 'epl_value' : 'credits' }, inplace=True)
final_data.sort_values(by="pos_detail",ascending=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,name,credits,pos_detail,club,mins_played_dream11,dream11_points,mins_played_league,dream_points,is_playing
16,Kieran Trippier,8.0,defender,ENG,90,42,2477,988.0,1
19,Ben Chilwell,8.0,defender,ENG,0,0,2290,1390.8,-1
15,Harry Maguire,8.0,defender,ENG,90,43,3048,1381.4,1
14,Reece James,8.5,defender,ENG,90,50,2367,1353.2,0
13,Antonio Rüdiger,8.5,defender,GER,270,59,1710,883.6,1
12,Kyle Walker,8.5,defender,ENG,180,67,1947,759.6,1
11,Matthias Ginter,8.5,defender,GER,259,73,3060,1309.8,1
10,Tyrone Mings,8.5,defender,ENG,191,96,3195,1102.8,0
17,Marcel Halstenberg,8.0,defender,GER,28,4,1653,995.4,0
9,Luke Shaw,8.5,defender,ENG,180,96,2657,1204.8,1


#INCLUDING ONLY PLAYERS IN THE LINE UP

In [251]:
final_data = final_data[final_data['is_playing']==1]

In [252]:
final_data.reset_index(inplace=True,drop=True)
final_data

,name,credits,pos_detail,club,mins_played_dream11,dream11_points,mins_played_league,dream_points,is_playing
0,Jordan Pickford,8.5,goalkeeper,ENG,270,108,2743,210.6,1
1,Manuel Neuer,8.5,goalkeeper,GER,270,33,2970,232.4,1
2,Robin Gosens,9.0,defender,GER,231,178,2539,2475.0,1
3,John Stones,9.0,defender,ENG,259,130,1934,967.4,1
4,Mats Hummels,8.5,defender,GER,243,118,2817,2100.2,1
5,Luke Shaw,8.5,defender,ENG,180,96,2657,1204.8,1
6,Matthias Ginter,8.5,defender,GER,259,73,3060,1309.8,1
7,Kyle Walker,8.5,defender,ENG,180,67,1947,759.6,1
8,Antonio Rüdiger,8.5,defender,GER,270,59,1710,883.6,1
9,Harry Maguire,8.0,defender,ENG,90,43,3048,1381.4,1


# ADDING WEIGHTS


100 DREAM, 0 LEAGUE

In [253]:
D100L0 = final_data.copy()
D100L0['ppm'] = ((1*final_data['dream11_points']) + (0*final_data['dream_points'])) / ((1*(final_data['mins_played_dream11'])/90) + (0*((final_data['mins_played_league'])/90)))
D100L0['ppm'].fillna(0)
D100L0

,name,credits,pos_detail,club,mins_played_dream11,dream11_points,mins_played_league,dream_points,is_playing,ppm
0,Jordan Pickford,8.5,goalkeeper,ENG,270,108,2743,210.6,1,36.000000
1,Manuel Neuer,8.5,goalkeeper,GER,270,33,2970,232.4,1,11.000000
2,Robin Gosens,9.0,defender,GER,231,178,2539,2475.0,1,69.350649
3,John Stones,9.0,defender,ENG,259,130,1934,967.4,1,45.173745
4,Mats Hummels,8.5,defender,GER,243,118,2817,2100.2,1,43.703704
5,Luke Shaw,8.5,defender,ENG,180,96,2657,1204.8,1,48.000000
6,Matthias Ginter,8.5,defender,GER,259,73,3060,1309.8,1,25.366795
7,Kyle Walker,8.5,defender,ENG,180,67,1947,759.6,1,33.500000
8,Antonio Rüdiger,8.5,defender,GER,270,59,1710,883.6,1,19.666667
9,Harry Maguire,8.0,defender,ENG,90,43,3048,1381.4,1,43.000000


In [254]:
#lp for finding team
data = D100L0.copy();
player = {str(i): data['name'][i] for i in range(data.shape[0])}
point = {str(i): data['ppm'][i] for i in range(data.shape[0])} 
cost = {str(i): data['credits'][i] for i in range(data.shape[0])}
gk = {str(i): 1 if data['pos_detail'][i] == 'goalkeeper' else 0 for i in range(data.shape[0])}
defender = {str(i): 1 if data['pos_detail'][i] == 'defender' else 0 for i in range(data.shape[0])}
midfielder = {str(i): 1 if data['pos_detail'][i] == 'midfielder' else 0 for i in range(data.shape[0])}
striker = {str(i): 1 if data['pos_detail'][i] == 'striker' else 0 for i in range(data.shape[0])}
club1 = {str(i): 1 if data['club'][i] == club1 else 0 for i in range(data.shape[0])}
club2 = {str(i): 1 if data['club'][i] == club2 else 0 for i in range(data.shape[0])}
xi = {str(i): 1 for i in range(data.shape[0])}

prob = LpProblem("Fantasy Football",LpMaximize)

player_index = LpVariable.dicts("Players",player,0,1,LpBinary)

# objective function
prob += lpSum([point[i]*player_index[i] for i in player]), "Total Cost"

# constraints
prob += lpSum([player_index[i] for i in player]) == 11, "Total 11 Players"
prob += lpSum([cost[i] * player_index[i] for i in player]) <= 100.0, "Total Cost"
prob += lpSum([gk[i] * player_index[i] for i in player]) == 1, "Only 1 GK"
prob += lpSum([defender[i] * player_index[i] for i in player]) >= 3, "Min 3 DEF"
prob += lpSum([defender[i] * player_index[i] for i in player]) <= 5, "Max 5 DEF"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) >= 3, "Min 3 MID"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) <= 5, "Max 5 MID"
prob += lpSum([striker[i] * player_index[i] for i in player]) >= 1, "Min 1 STR"
prob += lpSum([striker[i] * player_index[i] for i in player]) <= 3, "Max 3 STR"
prob += lpSum([club1[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club1"
prob += lpSum([club2[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club2"

# solve
status = prob.solve()
print(status)

1


/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [255]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [256]:
selection = {}
for v in prob.variables():
    index = int(v.name.split("_")[1])
    selection[index] = v.varValue

In [257]:
data['integer_programming'] = 0.0
for i in selection:
    data.loc[i, 'integer_programming'] = selection[i]

In [258]:
XI = data[data['integer_programming'] == 1.0]
TOP2 = XI['ppm'].nlargest(2)
CAPTAIN_POINTS = TOP2.iloc[0]
CAPTAIN = XI['name'][XI.ppm==CAPTAIN_POINTS]
VICECAPTAIN_POINTS = TOP2.iloc[1]
VICECAPTAIN = XI['name'][XI.ppm==VICECAPTAIN_POINTS]
PLAYER_POINTS = XI['ppm'].sum() 
TOTAL_POINTS = PLAYER_POINTS + CAPTAIN_POINTS + VICECAPTAIN_POINTS/2
TOTAL_COST = XI['credits'].sum()
TOTAL_PLAYERS = XI.shape[0]
print("Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

Total Points: 782.2550021913075, Raw Player Points: 617.1715142506768, Cost: 99.0, Captain: Leon Goretzka, Vice-Captain: Robin Gosens


In [259]:
print("100 DREAM : 0 LEAGUE")
print(XI[['name','pos_detail','club','credits','ppm',]]
 .sort_values(['pos_detail'])
 .reset_index(drop=True))

print("\n Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

100 DREAM : 0 LEAGUE
               name  pos_detail club  credits         ppm
0      Robin Gosens    defender  GER      9.0   69.350649
1       John Stones    defender  ENG      9.0   45.173745
2      Mats Hummels    defender  GER      8.5   43.703704
3         Luke Shaw    defender  ENG      8.5   48.000000
4   Jordan Pickford  goalkeeper  ENG      8.5   36.000000
5        Toni Kroos  midfielder  GER      9.5   43.333333
6    Joshua Kimmich  midfielder  GER      9.0   36.666667
7     Leon Goretzka  midfielder  GER      8.5  130.408163
8   Raheem Sterling     striker  ENG     10.0   50.487805
9       Kai Havertz     striker  GER      9.5   56.355140
10      Timo Werner     striker  GER      9.0   57.692308

 Total Points: 782.2550021913075, Raw Player Points: 617.1715142506768, Cost: 99.0, Captain: Leon Goretzka, Vice-Captain: Robin Gosens


0 DREAM 100 LEAGUE


In [260]:
D0L100 = final_data.copy()
D0L100['ppm'] = ((0*final_data['dream11_points']) + (1*final_data['dream_points'])) / ((0*(final_data['mins_played_dream11'])/90) + (1*((final_data['mins_played_league'])/90)))
D0L100['ppm'].fillna(0)
D0L100

,name,credits,pos_detail,club,mins_played_dream11,dream11_points,mins_played_league,dream_points,is_playing,ppm
0,Jordan Pickford,8.5,goalkeeper,ENG,270,108,2743,210.6,1,6.909953
1,Manuel Neuer,8.5,goalkeeper,GER,270,33,2970,232.4,1,7.042424
2,Robin Gosens,9.0,defender,GER,231,178,2539,2475.0,1,87.731390
3,John Stones,9.0,defender,ENG,259,130,1934,967.4,1,45.018614
4,Mats Hummels,8.5,defender,GER,243,118,2817,2100.2,1,67.099042
5,Luke Shaw,8.5,defender,ENG,180,96,2657,1204.8,1,40.809936
6,Matthias Ginter,8.5,defender,GER,259,73,3060,1309.8,1,38.523529
7,Kyle Walker,8.5,defender,ENG,180,67,1947,759.6,1,35.112481
8,Antonio Rüdiger,8.5,defender,GER,270,59,1710,883.6,1,46.505263
9,Harry Maguire,8.0,defender,ENG,90,43,3048,1381.4,1,40.789370


In [261]:
#lp for finding team
data = D0L100.copy();
player = {str(i): data['name'][i] for i in range(data.shape[0])}
point = {str(i): data['ppm'][i] for i in range(data.shape[0])} 
cost = {str(i): data['credits'][i] for i in range(data.shape[0])}
gk = {str(i): 1 if data['pos_detail'][i] == 'goalkeeper' else 0 for i in range(data.shape[0])}
defender = {str(i): 1 if data['pos_detail'][i] == 'defender' else 0 for i in range(data.shape[0])}
midfielder = {str(i): 1 if data['pos_detail'][i] == 'midfielder' else 0 for i in range(data.shape[0])}
striker = {str(i): 1 if data['pos_detail'][i] == 'striker' else 0 for i in range(data.shape[0])}
club1 = {str(i): 1 if data['club'][i] == club1 else 0 for i in range(data.shape[0])}
club2 = {str(i): 1 if data['club'][i] == club2 else 0 for i in range(data.shape[0])}
xi = {str(i): 1 for i in range(data.shape[0])}

prob = LpProblem("Fantasy Football",LpMaximize)

player_index = LpVariable.dicts("Players",player,0,1,LpBinary)

# objective function
prob += lpSum([point[i]*player_index[i] for i in player]), "Total Cost"

# constraints
prob += lpSum([player_index[i] for i in player]) == 11, "Total 11 Players"
prob += lpSum([cost[i] * player_index[i] for i in player]) <= 100.0, "Total Cost"
prob += lpSum([gk[i] * player_index[i] for i in player]) == 1, "Only 1 GK"
prob += lpSum([defender[i] * player_index[i] for i in player]) >= 3, "Min 3 DEF"
prob += lpSum([defender[i] * player_index[i] for i in player]) <= 5, "Max 5 DEF"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) >= 3, "Min 3 MID"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) <= 5, "Max 5 MID"
prob += lpSum([striker[i] * player_index[i] for i in player]) >= 1, "Min 1 STR"
prob += lpSum([striker[i] * player_index[i] for i in player]) <= 3, "Max 3 STR"
prob += lpSum([club1[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club1"
prob += lpSum([club2[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club2"

# solve
status = prob.solve()
print(status)

1


/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [262]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [263]:
selection = {}
for v in prob.variables():
    index = int(v.name.split("_")[1])
    selection[index] = v.varValue

In [264]:
data['integer_programming'] = 0.0
for i in selection:
    data.loc[i, 'integer_programming'] = selection[i]

In [265]:
XI = data[data['integer_programming'] == 1.0]
TOP2 = XI['ppm'].nlargest(2)
CAPTAIN_POINTS = TOP2.iloc[0]
CAPTAIN = XI['name'][XI.ppm==CAPTAIN_POINTS]
VICECAPTAIN_POINTS = TOP2.iloc[1]
VICECAPTAIN = XI['name'][XI.ppm==VICECAPTAIN_POINTS]
PLAYER_POINTS = XI['ppm'].sum() 
TOTAL_POINTS = PLAYER_POINTS + CAPTAIN_POINTS + VICECAPTAIN_POINTS/2
TOTAL_COST = XI['credits'].sum()
TOTAL_PLAYERS = XI.shape[0]
print("Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

Total Points: 973.1728204722259, Raw Player Points: 808.7065422262739, Cost: 99.5, Captain: Harry Kane, Vice-Captain: Robin Gosens


In [266]:
print("0 DREAM : 100 LEAGUE")
print(XI[['name','pos_detail','club','credits','ppm',]]
 .sort_values(['pos_detail'])
 .reset_index(drop=True))

print("\n Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

0 DREAM : 100 LEAGUE
               name  pos_detail club  credits         ppm
0      Robin Gosens    defender  GER      9.0   87.731390
1      Mats Hummels    defender  GER      8.5   67.099042
2   Antonio Rüdiger    defender  GER      8.5   46.505263
3      Manuel Neuer  goalkeeper  GER      8.5    7.042424
4        Toni Kroos  midfielder  GER      9.5   73.849199
5    Joshua Kimmich  midfielder  GER      9.0   79.611288
6     Thomas Müller  midfielder  GER      9.0   84.505404
7     Leon Goretzka  midfielder  GER      8.5   81.458699
8   Raheem Sterling     striker  ENG     10.0   82.181317
9        Harry Kane     striker  ENG     10.0  120.600583
10      Timo Werner     striker  GER      9.0   78.121933

 Total Points: 973.1728204722259, Raw Player Points: 808.7065422262739, Cost: 99.5, Captain: Harry Kane, Vice-Captain: Robin Gosens


50 DREAM 50 LEAGUE

In [267]:
D50L50 = final_data.copy()
D50L50['ppm'] = ((0.50*final_data['dream11_points']) + (0.50*final_data['dream_points'])) / ((0.50*(final_data['mins_played_dream11'])/90) + (0.50*((final_data['mins_played_league'])/90)))
D50L50['ppm'].fillna(0)
D50L50

,name,credits,pos_detail,club,mins_played_dream11,dream11_points,mins_played_league,dream_points,is_playing,ppm
0,Jordan Pickford,8.5,goalkeeper,ENG,270,108,2743,210.6,1,9.516761
1,Manuel Neuer,8.5,goalkeeper,GER,270,33,2970,232.4,1,7.372222
2,Robin Gosens,9.0,defender,GER,231,178,2539,2475.0,1,86.198556
3,John Stones,9.0,defender,ENG,259,130,1934,967.4,1,45.036936
4,Mats Hummels,8.5,defender,GER,243,118,2817,2100.2,1,65.241176
5,Luke Shaw,8.5,defender,ENG,180,96,2657,1204.8,1,41.266126
6,Matthias Ginter,8.5,defender,GER,259,73,3060,1309.8,1,37.496836
7,Kyle Walker,8.5,defender,ENG,180,67,1947,759.6,1,34.976023
8,Antonio Rüdiger,8.5,defender,GER,270,59,1710,883.6,1,42.845455
9,Harry Maguire,8.0,defender,ENG,90,43,3048,1381.4,1,40.852772


In [268]:
#lp for finding team
data = D50L50.copy();
player = {str(i): data['name'][i] for i in range(data.shape[0])}
point = {str(i): data['ppm'][i] for i in range(data.shape[0])} 
cost = {str(i): data['credits'][i] for i in range(data.shape[0])}
gk = {str(i): 1 if data['pos_detail'][i] == 'goalkeeper' else 0 for i in range(data.shape[0])}
defender = {str(i): 1 if data['pos_detail'][i] == 'defender' else 0 for i in range(data.shape[0])}
midfielder = {str(i): 1 if data['pos_detail'][i] == 'midfielder' else 0 for i in range(data.shape[0])}
striker = {str(i): 1 if data['pos_detail'][i] == 'striker' else 0 for i in range(data.shape[0])}
club1 = {str(i): 1 if data['club'][i] == club1 else 0 for i in range(data.shape[0])}
club2 = {str(i): 1 if data['club'][i] == club2 else 0 for i in range(data.shape[0])}
xi = {str(i): 1 for i in range(data.shape[0])}

prob = LpProblem("Fantasy Football",LpMaximize)

player_index = LpVariable.dicts("Players",player,0,1,LpBinary)

# objective function
prob += lpSum([point[i]*player_index[i] for i in player]), "Total Cost"

# constraints
prob += lpSum([player_index[i] for i in player]) == 11, "Total 11 Players"
prob += lpSum([cost[i] * player_index[i] for i in player]) <= 100.0, "Total Cost"
prob += lpSum([gk[i] * player_index[i] for i in player]) == 1, "Only 1 GK"
prob += lpSum([defender[i] * player_index[i] for i in player]) >= 3, "Min 3 DEF"
prob += lpSum([defender[i] * player_index[i] for i in player]) <= 5, "Max 5 DEF"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) >= 3, "Min 3 MID"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) <= 5, "Max 5 MID"
prob += lpSum([striker[i] * player_index[i] for i in player]) >= 1, "Min 1 STR"
prob += lpSum([striker[i] * player_index[i] for i in player]) <= 3, "Max 3 STR"
prob += lpSum([club1[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club1"
prob += lpSum([club2[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club2"

# solve
status = prob.solve()
print(status)

1


/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [269]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [270]:
selection = {}
for v in prob.variables():
    index = int(v.name.split("_")[1])
    selection[index] = v.varValue

In [271]:
data['integer_programming'] = 0.0
for i in selection:
    data.loc[i, 'integer_programming'] = selection[i]

In [272]:
XI = data[data['integer_programming'] == 1.0]
TOP2 = XI['ppm'].nlargest(2)
CAPTAIN_POINTS = TOP2.iloc[0]
CAPTAIN = XI['name'][XI.ppm==CAPTAIN_POINTS]
VICECAPTAIN_POINTS = TOP2.iloc[1]
VICECAPTAIN = XI['name'][XI.ppm==VICECAPTAIN_POINTS]
PLAYER_POINTS = XI['ppm'].sum() 
TOTAL_POINTS = PLAYER_POINTS + CAPTAIN_POINTS + VICECAPTAIN_POINTS/2
TOTAL_COST = XI['credits'].sum()
TOTAL_PLAYERS = XI.shape[0]
print("Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

Total Points: 939.5962583660302, Raw Player Points: 783.9605267423262, Cost: 100.0, Captain: Harry Kane, Vice-Captain: Robin Gosens


In [273]:
print("50 DREAM : 50 LEAGUE \n")
print(XI[['name','pos_detail','club','credits','ppm',]]
 .sort_values(['pos_detail'])
 .reset_index(drop=True))

print("\n Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

50 DREAM : 50 LEAGUE 

               name  pos_detail club  credits         ppm
0      Robin Gosens    defender  GER      9.0   86.198556
1       John Stones    defender  ENG      9.0   45.036936
2      Mats Hummels    defender  GER      8.5   65.241176
3   Jordan Pickford  goalkeeper  ENG      8.5    9.516761
4        Toni Kroos  midfielder  GER      9.5   70.404682
5    Joshua Kimmich  midfielder  GER      9.0   74.911228
6     Thomas Müller  midfielder  GER      9.0   80.089397
7     Leon Goretzka  midfielder  GER      8.5   82.824601
8   Raheem Sterling     striker  ENG     10.0   79.379806
9        Harry Kane     striker  ENG     10.0  112.536454
10      Timo Werner     striker  GER      9.0   77.820929

 Total Points: 939.5962583660302, Raw Player Points: 783.9605267423262, Cost: 100.0, Captain: Harry Kane, Vice-Captain: Robin Gosens


75 DREAM 25 LEAGUE

In [274]:
D75L25 = final_data.copy()
D75L25['ppm'] = ((0.75*final_data['dream11_points']) + (0.25*final_data['dream_points'])) / ((0.75*(final_data['mins_played_dream11'])/90) + (0.25*((final_data['mins_played_league'])/90)))
D75L25['ppm'].fillna(0)
D75L25

,name,credits,pos_detail,club,mins_played_dream11,dream11_points,mins_played_league,dream_points,is_playing,ppm
0,Jordan Pickford,8.5,goalkeeper,ENG,270,108,2743,210.6,1,13.541796
1,Manuel Neuer,8.5,goalkeeper,GER,270,33,2970,232.4,1,7.890476
2,Robin Gosens,9.0,defender,GER,231,178,2539,2475.0,1,83.790223
3,John Stones,9.0,defender,ENG,259,130,1934,967.4,1,45.063076
4,Mats Hummels,8.5,defender,GER,243,118,2817,2100.2,1,62.289340
5,Luke Shaw,8.5,defender,ENG,180,96,2657,1204.8,1,42.024398
6,Matthias Ginter,8.5,defender,GER,259,73,3060,1309.8,1,35.859265
7,Kyle Walker,8.5,defender,ENG,180,67,1947,759.6,1,34.762364
8,Antonio Rüdiger,8.5,defender,GER,270,59,1710,883.6,1,37.878571
9,Harry Maguire,8.0,defender,ENG,90,43,3048,1381.4,1,40.969259


In [275]:
#lp for finding team
data = D75L25.copy();
player = {str(i): data['name'][i] for i in range(data.shape[0])}
point = {str(i): data['ppm'][i] for i in range(data.shape[0])} 
cost = {str(i): data['credits'][i] for i in range(data.shape[0])}
gk = {str(i): 1 if data['pos_detail'][i] == 'goalkeeper' else 0 for i in range(data.shape[0])}
defender = {str(i): 1 if data['pos_detail'][i] == 'defender' else 0 for i in range(data.shape[0])}
midfielder = {str(i): 1 if data['pos_detail'][i] == 'midfielder' else 0 for i in range(data.shape[0])}
striker = {str(i): 1 if data['pos_detail'][i] == 'striker' else 0 for i in range(data.shape[0])}
club1 = {str(i): 1 if data['club'][i] == club1 else 0 for i in range(data.shape[0])}
club2 = {str(i): 1 if data['club'][i] == club2 else 0 for i in range(data.shape[0])}
xi = {str(i): 1 for i in range(data.shape[0])}

prob = LpProblem("Fantasy Football",LpMaximize)

player_index = LpVariable.dicts("Players",player,0,1,LpBinary)

# objective function
prob += lpSum([point[i]*player_index[i] for i in player]), "Total Cost"

# constraints
prob += lpSum([player_index[i] for i in player]) == 11, "Total 11 Players"
prob += lpSum([cost[i] * player_index[i] for i in player]) <= 100.0, "Total Cost"
prob += lpSum([gk[i] * player_index[i] for i in player]) == 1, "Only 1 GK"
prob += lpSum([defender[i] * player_index[i] for i in player]) >= 3, "Min 3 DEF"
prob += lpSum([defender[i] * player_index[i] for i in player]) <= 5, "Max 5 DEF"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) >= 3, "Min 3 MID"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) <= 5, "Max 5 MID"
prob += lpSum([striker[i] * player_index[i] for i in player]) >= 1, "Min 1 STR"
prob += lpSum([striker[i] * player_index[i] for i in player]) <= 3, "Max 3 STR"
prob += lpSum([club1[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club1"
prob += lpSum([club2[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club2"

# solve
status = prob.solve()
print(status)

1


/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [276]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [277]:
selection = {}
for v in prob.variables():
    index = int(v.name.split("_")[1])
    selection[index] = v.varValue

In [278]:
data['integer_programming'] = 0.0
for i in selection:
    data.loc[i, 'integer_programming'] = selection[i]

In [279]:
XI = data[data['integer_programming'] == 1.0]
TOP2 = XI['ppm'].nlargest(2)
CAPTAIN_POINTS = TOP2.iloc[0]
CAPTAIN = XI['name'][XI.ppm==CAPTAIN_POINTS]
VICECAPTAIN_POINTS = TOP2.iloc[1]
VICECAPTAIN = XI['name'][XI.ppm==VICECAPTAIN_POINTS]
PLAYER_POINTS = XI['ppm'].sum() 
TOTAL_POINTS = PLAYER_POINTS + CAPTAIN_POINTS + VICECAPTAIN_POINTS/2
TOTAL_COST = XI['credits'].sum()
TOTAL_PLAYERS = XI.shape[0]
print("Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

Total Points: 890.3716143039256, Raw Player Points: 748.1817113913042, Cost: 100.0, Captain: Harry Kane, Vice-Captain: Leon Goretzka


In [280]:
print("75 DREAM : 25 LEAGUE \n")
print(XI[['name','pos_detail','club','credits','ppm',]]
 .sort_values(['pos_detail'])
 .reset_index(drop=True))

print("\n Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

75 DREAM : 25 LEAGUE 

               name  pos_detail club  credits        ppm
0      Robin Gosens    defender  GER      9.0  83.790223
1       John Stones    defender  ENG      9.0  45.063076
2      Mats Hummels    defender  GER      8.5  62.289340
3   Jordan Pickford  goalkeeper  ENG      8.5  13.541796
4        Toni Kroos  midfielder  GER      9.5  65.418827
5    Joshua Kimmich  midfielder  GER      9.0  68.043232
6     Thomas Müller  midfielder  GER      9.0  72.891252
7     Leon Goretzka  midfielder  GER      8.5  85.339806
8   Raheem Sterling     striker  ENG     10.0  75.039389
9        Harry Kane     striker  ENG     10.0  99.520000
10      Timo Werner     striker  GER      9.0  77.244771

 Total Points: 890.3716143039256, Raw Player Points: 748.1817113913042, Cost: 100.0, Captain: Harry Kane, Vice-Captain: Leon Goretzka


DREAM 25 LEAGUE 75

In [281]:
D25L75 = final_data.copy()
D25L75['ppm'] = ((0.25*final_data['dream11_points']) + (0.75*final_data['dream_points'])) / ((0.25*(final_data['mins_played_dream11'])/90) + (0.75*((final_data['mins_played_league'])/90)))
D25L75['ppm'].fillna(0)
D25L75

,name,credits,pos_detail,club,mins_played_dream11,dream11_points,mins_played_league,dream_points,is_playing,ppm
0,Jordan Pickford,8.5,goalkeeper,ENG,270,108,2743,210.6,1,7.834098
1,Manuel Neuer,8.5,goalkeeper,GER,270,33,2970,232.4,1,7.158824
2,Robin Gosens,9.0,defender,GER,231,178,2539,2475.0,1,87.190367
3,John Stones,9.0,defender,ENG,259,130,1934,967.4,1,45.025243
4,Mats Hummels,8.5,defender,GER,243,118,2817,2100.2,1,66.445135
5,Luke Shaw,8.5,defender,ENG,180,96,2657,1204.8,1,40.968715
6,Matthias Ginter,8.5,defender,GER,259,73,3060,1309.8,1,38.162517
7,Kyle Walker,8.5,defender,ENG,180,67,1947,759.6,1,35.064275
8,Antonio Rüdiger,8.5,defender,GER,270,59,1710,883.6,1,45.163333
9,Harry Maguire,8.0,defender,ENG,90,43,3048,1381.4,1,40.810916


In [282]:
#lp for finding team
data = D25L75.copy();
player = {str(i): data['name'][i] for i in range(data.shape[0])}
point = {str(i): data['ppm'][i] for i in range(data.shape[0])} 
cost = {str(i): data['credits'][i] for i in range(data.shape[0])}
gk = {str(i): 1 if data['pos_detail'][i] == 'goalkeeper' else 0 for i in range(data.shape[0])}
defender = {str(i): 1 if data['pos_detail'][i] == 'defender' else 0 for i in range(data.shape[0])}
midfielder = {str(i): 1 if data['pos_detail'][i] == 'midfielder' else 0 for i in range(data.shape[0])}
striker = {str(i): 1 if data['pos_detail'][i] == 'striker' else 0 for i in range(data.shape[0])}
club1 = {str(i): 1 if data['club'][i] == club1 else 0 for i in range(data.shape[0])}
club2 = {str(i): 1 if data['club'][i] == club2 else 0 for i in range(data.shape[0])}
xi = {str(i): 1 for i in range(data.shape[0])}

prob = LpProblem("Fantasy Football",LpMaximize)

player_index = LpVariable.dicts("Players",player,0,1,LpBinary)

# objective function
prob += lpSum([point[i]*player_index[i] for i in player]), "Total Cost"

# constraints
prob += lpSum([player_index[i] for i in player]) == 11, "Total 11 Players"
prob += lpSum([cost[i] * player_index[i] for i in player]) <= 100.0, "Total Cost"
prob += lpSum([gk[i] * player_index[i] for i in player]) == 1, "Only 1 GK"
prob += lpSum([defender[i] * player_index[i] for i in player]) >= 3, "Min 3 DEF"
prob += lpSum([defender[i] * player_index[i] for i in player]) <= 5, "Max 5 DEF"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) >= 3, "Min 3 MID"
prob += lpSum([midfielder[i] * player_index[i] for i in player]) <= 5, "Max 5 MID"
prob += lpSum([striker[i] * player_index[i] for i in player]) >= 1, "Min 1 STR"
prob += lpSum([striker[i] * player_index[i] for i in player]) <= 3, "Max 3 STR"
prob += lpSum([club1[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club1"
prob += lpSum([club2[i] * player_index[i] for i in player]) <=7, "Maximum 7 Players from club2"

# solve
status = prob.solve()
print(status)

1


/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [283]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [284]:
selection = {}
for v in prob.variables():
    index = int(v.name.split("_")[1])
    selection[index] = v.varValue

In [285]:
data['integer_programming'] = 0.0
for i in selection:
    data.loc[i, 'integer_programming'] = selection[i]

In [286]:
XI = data[data['integer_programming'] == 1.0]
TOP2 = XI['ppm'].nlargest(2)
CAPTAIN_POINTS = TOP2.iloc[0]
CAPTAIN = XI['name'][XI.ppm==CAPTAIN_POINTS]
VICECAPTAIN_POINTS = TOP2.iloc[1]
VICECAPTAIN = XI['name'][XI.ppm==VICECAPTAIN_POINTS]
PLAYER_POINTS = XI['ppm'].sum() 
TOTAL_POINTS = PLAYER_POINTS + CAPTAIN_POINTS + VICECAPTAIN_POINTS/2
TOTAL_COST = XI['credits'].sum()
TOTAL_PLAYERS = XI.shape[0]
print("Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

Total Points: 960.397081891031, Raw Player Points: 799.0284683006587, Cost: 99.5, Captain: Harry Kane, Vice-Captain: Robin Gosens


In [287]:
print("25 DREAM : 75 LEAGUE \n")
print(XI[['name','pos_detail','club','credits','ppm',]]
 .sort_values(['pos_detail'])
 .reset_index(drop=True))

print("\n Total Points: {}, Raw Player Points: {}, Cost: {}, Captain: {}, Vice-Captain: {}".format(TOTAL_POINTS, PLAYER_POINTS, TOTAL_COST, CAPTAIN.iloc[0],VICECAPTAIN.iloc[0]))

25 DREAM : 75 LEAGUE 

               name  pos_detail club  credits         ppm
0      Robin Gosens    defender  GER      9.0   87.190367
1      Mats Hummels    defender  GER      8.5   66.445135
2   Antonio Rüdiger    defender  GER      8.5   45.163333
3   Jordan Pickford  goalkeeper  ENG      8.5    7.834098
4        Toni Kroos  midfielder  GER      9.5   72.607595
5    Joshua Kimmich  midfielder  GER      9.0   77.921294
6     Thomas Müller  midfielder  GER      9.0   82.960979
7     Leon Goretzka  midfielder  GER      8.5   81.922631
8   Raheem Sterling     striker  ENG     10.0   81.189003
9        Harry Kane     striker  ENG     10.0  117.773430
10      Timo Werner     striker  GER      9.0   78.020603

 Total Points: 960.397081891031, Raw Player Points: 799.0284683006587, Cost: 99.5, Captain: Harry Kane, Vice-Captain: Robin Gosens
